## Data Peprocessing (word segmentation, keyword extracton, word normalisation)

In [1]:
import os
import csv
import json
from collections import Counter, defaultdict

### Load CKIP (download data if you haven't)

In [ ]:
#Uncomment line below. Downloads to ./data.zip (2GB) and extracts to ./data/
### data_utils.download_data_gdown("./") # gdrive-ckip

In [4]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import tensorflow
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

ws = WS("./data", disable_cuda=False)
pos = POS("./data", disable_cuda=False)
ner = NER("./data", disable_cuda=False)

/Users/melody/Documents/Courses 2021 autumn/Data Mining/labs/lib/python3.9/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/Users/melody/Documents/Courses 2021 autumn/Data Mining/labs/lib/python3.9/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
/Users/melody/Documents/Courses 2021 autumn/Data Mining/labs/lib/python3.9/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(
2021-12-04 

#### Preprocessed keywords
... or just use the preprocessed files (skip 3 cells below, find+use those with `.jsonl` extension)

In [ ]:
from collections import defaultdict

keyword_pth = "../Keywords"
kw_files = defaultdict(lambda: list())

for filename in os.listdir(keyword_pth):
    print(filename)
    if filename.split(".")[-1] == "xlsx":
        continue
    with open(os.path.join(keyword_pth, filename), newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            kw_files[filename].append(row)

In [37]:
kw_files_cleaned = defaultdict(lambda: list())

for filename, file_cont in kw_files.items():
    for name_lst in file_cont:  
        new_names = []
        for name in name_lst:
            if name.strip():
                new_names.append(name)
        kw_files_cleaned[filename].append(new_names.copy())
        
kw_files_cleaned['02chem.list.csv']

In [51]:
for filename in kw_files_cleaned.keys():
    file_newname = ".".join(filename.split(".")[:-1])
    file_newname = file_newname+".jsonl"
    with jsonlines.open(file_newname, mode='w') as writer:
        writer.write(kw_files_cleaned[filename])     

<< **Load keyword files here.** >>

In [34]:
import jsonlines

kwords = []
for filename in os.listdir("./Keywords"):
    if filename.split(".")[-1]=="jsonl":
        kwords.extend([line for line in jsonlines.open("./Keywords/"+filename)])

In [35]:
kw_all = []

for type_ in kwords: # crop, pest, chem
    for it in type_:
        for i in it:
            kw_all.append(i)
kw_all = set(kw_all)

Make dictionaries for text normalisation:
- `synonym_dict`: `{normalised keyword: [other keywords (to be replaced)]}`
    - Just load `synonym_dict.json` 3 cells below for you convenience
- `inv_synonym_dict`: inverted `synonym_dict` to perform the replacing (i.e. values become keys and vice versa)

In [114]:
synonym_dict = {}
synonym_set = set()

for filename, filecont in kw_files_cleaned.items():
    for namelist in filecont:
        if len(namelist) > 1:
            synonym_dict[namelist[0]] = namelist[1:]
            synonym_set  = synonym_set | set(namelist[1:])

In [102]:
list(synonym_dict.items())[5]

('斑飛蝨', {'穗苔'})

In [117]:
with open("synonym_dict.json", "w") as f:
    json.dump(synonym_dict, f)

<< ** Load `synonym_dict` here ** >>

In [98]:
with open("synonym_dict.json") as f:
    synonym_dict = json.load(f)

inv_synonym_dict = dict()

for k, v1 in synonym_dict.items():
    for v in v1: 
        inv_synonym_dict[v] = k
    inv_synonym_dict[k] = k

### Create coerce & recommend dictionary from keywords for ckip 
(code from group member)

In [7]:
import pandas as pd
Path = Path = './Keywords/'
### create a coerce dictionary by the pest list and the chem list
dfpest = pd.read_excel(Path+'02pest.list.xlsx', engine='openpyxl', header=None)
dfchem = pd.read_excel(Path+'02chem.list.xlsx', engine='openpyxl', header=None)

coerce_dict = pd.concat([dfpest, dfchem]).to_numpy().reshape(-1).tolist()
coerce_dict = construct_dictionary(
    dict((word, 1) for word in coerce_dict if pd.isnull(word) == False)
    )

### create a recommend dictionary only by the crop list
dfcrop = pd.read_excel(Path+'02crop.list.xlsx', engine='openpyxl', header=None)
recommend_dict = dfcrop.to_numpy().reshape(-1).tolist()
recommend_dict = construct_dictionary(
    dict((word, 1) for word in recommend_dict if pd.isnull(word) == False)
    )

### Word segmentation + extract keywords for each article
You may also just use the preprocessed file `doc_keywords.json` 3 cells below.

In [ ]:
from tqdm import tqdm
import os

Path = './dataTrainComplete/'
filelist = os.listdir(Path)
doc_kw_dict = dict()

for file in tqdm(filelist):
    if file[0]=='.':
        continue
    f = open(Path+file, 'r', encoding='utf-8')
    article = "".join([line.strip() for line in f.readlines()])
    article = re.sub('[0-9]+', "N", article)
    article = re.sub('[a-zA-Z]+', "A", article)
    article = re.sub('[^\u4e00-\u9fa5\uff0c\u3001\u3002\uff1bNA]+|\u7684','', article)
#     artilist.append(re.sub('[0-9]+|\x7f|\n|\!|\'|\?|\）|\（|\、|\。','', f.read()))
    seg_res = ws([article], recommend_dictionary=recommend_dict, coerce_dictionary=coerce_dict)  # 进行分词
    seg_res = flattenList(seg_res)
    kws = get_keywords(kw_all, seg_res)
    
    docnum = int(file.split(".")[0])
    doc_kw_dict[docnum] = kws.copy()
    
    f.close()

In [70]:
def flattenList(nestedList):
     # source:  geeks2geeks
    # check if list is empty
    if not(bool(nestedList)):
        return nestedList
 
     # to check instance of list is empty or not
    # a list is an instance of list
    # a str, int, etc., is not an instance of list, and will therefore be returned as the final line of this func
    if isinstance(nestedList[0], list):
 
        # call function with sublist as argument
        return flattenList(*nestedList[:1]) + flattenList(nestedList[1:])
 
    # call function with sublist as argument
    return nestedList[:1] + flattenList(nestedList[1:])

In [64]:
def get_keywords(all_kw_set, seg_res):
    """
    seg_res = segmentation results from ckip (output of ws)
    """
    kw_lst = []
    for w in seg_res:
        if w in all_kw_set:
            kw_lst.append(w)
    return kw_lst

In [85]:
with open("doc_keywords.json", 'w') as f:
    json.dump(doc_kws, f)

<< **Load `doc_keywords` here** >>

In [132]:
import json

# get keywords for each document
with open("doc_keywords.json") as f:
    doc_kws_ = json.load(f)

# turn keys from string to integer
doc_kws = dict()
for k in doc_kws_.keys():
    doc_kws[int(k)] = doc_kws_[k].copy()

### Build data structure for storing docs with keywords 
Prob not necessary in hindsight, built this mainly for the graph thing but used it anyway. 

In [140]:
class Node:
    def __init__(self, node_num, kwords):
        # when calling Node(in_nodes, out_nodes, kwords), __init__ will be executed
        """
        node_num: document id
        kwords: list of strings; represents the keywords that the node article contains
        """
        self.node_num = node_num
        self.kwords = kwords
    

In [141]:
nodes = dict()

for doc in doc_id_sorted:
    cur_kws = set()
    for w in doc_kws[doc]:
        if w in inv_synonym_dict.keys(): # 需替換
            cur_kws.add(inv_synonym_dict[w])
        else: # 為本名/無別名，不需替換
            cur_kws.add(w)
    node = Node(doc, cur_kws)
    nodes[doc] = node

---
### ...finally the classification based on intersection of keyword sets

In [265]:
# test on training data
ans = dict()
kws = []
ratios = []
intersec_ratio = 1
kw_ratio = 0.4 # start with lower values e.g. 0.2 and scale up 
#lower_ratio = 0.4

for doc in nodes.values():
    for doc_cmp in nodes.values():
        doc_kw = doc.kwords
        cmp_kw = doc_cmp.kwords
        intersec = doc_kw & cmp_kw
        if len(doc_kw) and doc.node_num != doc_cmp.node_num:
            intersec_ratio = len(intersec)/len(doc_kw)
            if not bool(intersec):
                continue
            elif intersec_ratio >= intersec_ratio: # 兩文件的相似度
                if len(doc_kw)/len(cmp_kw) > kw_ratio: # 被比較的文件的 keywords 在另一文件中的佔比
                    ans_dict = {"kws": (doc_kw, cmp_kw), "ratios": intersec_ratio}
                    ans_key = str(doc.node_num)+"-"+str(doc_cmp.node_num)
                    ans[ans_key] = ans_dict
                # if len(doc_kw) > len(cmp_kw) => (cmp, doc) will be in ans but not (doc, cmp)
            #elif intersec_ratio > lower_ratio and intersec_ratio < intersec_ratio:
                #ans.append((doc.node_num, doc_cmp.node_num))
                #ans.append((doc_cmp.node_num, doc.node_num))
len(ans.keys())          

1727

---
### Evaluation

In [204]:
def precision(pred, gold):
    """
    TP/(TP+FP)
    """
    tp = 0
    fp = 0
    for p in pred:
        if p in gold:
            tp+=1
        else:
            fp+=1
    return tp/(tp+fp)

In [205]:
def recall(pred, gold):
    """
    TP/(TP+FN)
    """
    tp = 0
    fn = 0
    for g in gold:
        if g in pred:
            tp += 1
        else:
            fn+=1
    return tp/(tp+fn)

In [203]:
def F1(prec, rcl):
    return 2*prec*rcl/(prec+rcl)

In [45]:
csv_path = "TrainLabel.csv"
related_pairs = []

with open(csv_path, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    rows = list(spamreader)
    for it in rows[1:]: 
        related_pairs.append([int(it[0]), int(it[1])])

In [267]:
pred = ans.keys()
gold = [str(tp[0])+"-"+str(tp[1]) for tp in related_pairs]

precision_score = precision(pred, gold)
recall_score = recall(pred, gold)
f1_score = F1(precision_score, recall_score)

print("{:.2%}, {:.2%}, {:.2%}".format(precision_score, recall_score, f1_score))

---
### Read test data
Or load directly from ready-made file (3 cells below)

In [210]:
test_docs = dict()

for filename in os.listdir("./dataTestPublicComplete"):
    if filename[0] == ".":
        continue
    with open("./dataTestPublicComplete/"+filename) as f:
        test_docs[int(filename.split(".")[0])] = "".join([line.strip() for line in f.readlines()])

In [213]:
test_docs[1053]

'近來氣候高溫多濕，相當適合草莓苗期炭疽病發生，籲請農友注意防治適期。苗栗區農改場、防檢局及田邊好幫手關心您。近來氣候高溫多濕，相當適合草莓苗期炭疽病發生，籲請農友注意防治適期。防治應注意事項:(1)請加強母株的防治，若育苗區母株已呈現萎凋狀，請徹底清除勿留在園區。(2)去除老葉之傷口為炭疽病菌侵入冠部之重要關鍵，老葉去除後應施用藥劑，以防病菌感染。育苗區之葉片、走蔓若有炭疽病斑，請將較嚴重之葉片或走蔓去除，並於24小時內配合藥劑之噴施。(3)噴施藥劑時須特別注意冠部亦應有足夠之劑量，因冠部若腐壞會造成植株萎凋死亡。農友如果有任何技術上的疑問，可以直接洽詢苗栗區農業改良場諮詢專線037-236583或病蟲害診斷專線037-236619，或各鄉鎮市公所、農會。防治藥劑及方法：(1)53％腐絕快得寧可濕性粉劑1,200倍(2)24.9%待克利乳劑3,000倍(3)24.9%待克利水懸劑3,000倍(4)23.6%百克敏乳劑3,000倍'

In [214]:
with open("test_docs.json", 'w') as f:
    json.dump(test_docs, f)

<< ** Load `test_docs` here ** >>

In [ ]:
with open("test_docs.json") as f:
    test_docs = json.load(f)

### Preprocess keywords
( or skip 1 cell and just use the ready-made file)

In [103]:
# segmentation for test docs
test_doc_kw = dict()

for docnum, article in tqdm(test_docs.items()):
    article = re.sub('[0-9]+', "N", article)
    article = re.sub('[a-zA-Z]+', "A", article)
    article = re.sub('[^\u4e00-\u9fa5\uff0c\u3001\u3002\uff1bNA]+|\u7684','', article)
    #     artilist.append(re.sub('[0-9]+|\x7f|\n|\!|\'|\?|\）|\（|\、|\。','', f.read()))
    seg_res = ws([article], recommend_dictionary=recommend_dict, coerce_dictionary=coerce_dict)  # 进行分词
    seg_res = flattenList(seg_res)
    kws = get_keywords(kw_all, seg_res)
    test_doc_kw[docnum] = kws.copy()

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 421/421 [04:17<00:00,  1.63it/s]


In [225]:
with open("test_doc_kw.json") as f:
    test_doc_kws_ = json.load(f)

test_doc_kws = dict()
for k in test_doc_kws_.keys():
    test_doc_kws[int(k)] = test_doc_kws_[k].copy()

### Make data structure for storing doc and keyword info

In [226]:
test_nodes = dict()

for doc in test_docs.keys():
    test_cur_kws = set()
    for w in test_doc_kws[doc]:
        if w in inv_synonym_dict.keys(): # 需替換
            test_cur_kws.add(inv_synonym_dict[w])
        else: # 為本名/無別名，不需替換
            test_cur_kws.add(w)
    test_node = Node(doc, test_cur_kws)
    test_nodes[doc] = test_node

In [228]:
test_nodes[1053].kwords

{'待克利', '炭疽病', '百克敏', '腐絕快得寧', '草莓'}

In [274]:
# test on training data
test_ans = dict()
test_kws = []
test_ratios = []
intersec_ratio = 1
kw_ratio = 0.4
#lower_ratio = 0.4
no_intersec = dict() # for error analysis

for doc in test_nodes.values():
    for doc_cmp in test_nodes.values():
        doc_kw = doc.kwords
        cmp_kw = doc_cmp.kwords
        intersec = doc_kw & cmp_kw
        
        if len(doc_kw) and doc.node_num != doc_cmp.node_num:
            intersec_ratio = len(intersec)/len(doc_kw)
            if not bool(intersec):
                ni_dict = {"kws": (doc_kw, cmp_kw)}
                ni_key = str(doc.node_num)+"-"+str(doc_cmp.node_num)
                no_intersec[ni_key] = ni_dict
                continue
            elif intersec_ratio >= intersec_ratio: # 兩文件的相似度
                if len(doc_kw) / len(cmp_kw) > kw_ratio:
                    ans_dict = {"kws": (doc_kw, cmp_kw), "ratios": intersec_ratio}
                    ans_key = str(doc.node_num)+"-"+str(doc_cmp.node_num)
                    test_ans[ans_key] = ans_dict
                # if len(doc_kw) > len(cmp_kw) => (cmp, doc) will be in ans but not (doc, cmp)
            #elif intersec_ratio > lower_ratio and intersec_ratio < intersec_ratio:
                #ans.append((doc.node_num, doc_cmp.node_num))
                #ans.append((doc_cmp.node_num, doc.node_num))
len(test_ans.keys())          

615

In [275]:
import csv

test_ans_pairs = []
for k in test_ans.keys():
    test_ans_pairs.append(k.split("-"))
    
submit_num = 5
    
with open('submit{}.csv'.format(submit_num), 'w', newline='') as f:
    spamwriter = csv.writer(f, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["Test","Reference"])
    for k in test_ans_pairs:
        spamwriter.writerow(k)